# Comparing effects of DWI Preprocessing and WMQL queries on fiber bundle statistics

## Get Data

10 random INTRuST cases were picked, and WMQL was run under 8 scenarios:

1. Original DWI, matlab ukf, original WMQL query
2. Original DWI, matlab ukf, new WMQL query
3. Original DWI, c++ ukf, original WMQL query
4. Original DWI, c++ ukf, new WMQL query
5. Harmonized DWI, c++ ukf, original WMQL query
6. Harmonized DWI, c++ ukf, new WMQL query
7. Harmonized/EPI corrected DWI, c++ ukf, original WMQL query
8. Harmonized/EPI corrected DWI, c++ ukf, new WMQL query

(new WMQL query is Nikos/Peter's suggested change to improve e.g ioff fiber bundle results)

In [25]:
load'csv'
localpath=. '/Users/ryan/partners'&,  NB. get local path
download=. ([: shell 'clusterget.sh '&,)`1:@.(fexist@:localpath)
BASE=.'/data/pnl/intrust-wmql-experiments/'

o=.BASE,'original-matlabukf/tractmeasures.csv'
on=.BASE,'original-matlabukf-newquery/tractmeasures.csv'
ocpp=.BASE,'original-cppukf/tractmeasures.csv'
ocppn=.BASE,'original-cppukf-newquery/tractmeasures.csv'
h=.BASE,'harmonized-cppukf/tractmeasures.csv'
hn=.BASE,'harmonized-cppukf-newquery/tractmeasures.csv'
he=.BASE,'harmonizedepi-cppukf/tractmeasures.csv'
hen=.BASE,'harmonizedepi-cppukf-newquery/tractmeasures.csv'
download each REMOTE_PATHS=.o;on;ocpp;ocppn;h;hn;he;hen
csvs=. readcsv@:localpath each REMOTE_PATHS

In [24]:
download h

receiving file list ... done
data/pnl/intrust-wmql-experiments/harmonized-cppukf/
data/pnl/intrust-wmql-experiments/harmonized-cppukf/tractmeasures.csv

sent 48 bytes  received 217772 bytes  22928.42 bytes/sec
total size is 508689  speedup is 2.34



## Comparison

For each case and for each fiber bundle, we have mean FA, standard deviation of FA,
and number of tracts. Under each of the 6 DWI/WMQL scenarios listed above, we average over the 10 cases and calculate
4 statistics for each bundle:

1. Number of cases for which fiber bundle was found
2. Mean number of tracts (group by 'num_fibers-num_fibers' in tractmeasures.csv)
3. Average mean FA (group by 'FA-mean' in tractmeasures.csv)
4. Average standard deviation of FA (group by 'FA-stDev' in tractmeasures.csv)

In the boxes below, the rows are DWI/WMQL scenarios and the numbers are the 4 statistics.  The
rows are grouped by fiber bundle to compare the effect of each scenario on a fiber bundle's
statistics.

In [26]:
9!:37 [ 0 180 1000 1000
NB. Functions
load'regex'
mean=:+/ % #
getcol=:}.@:([ {"1~ (i.~ {.)~) 
tonum=. ".every
removeid=: ('^[[:digit:]]{3}_[[:digit:][:alpha:]]{3}_[[:digit:][:alpha:]]{3}_';'')&rxrplc
tractcol=. (removeid each)@:(getcol&(<'tract'))
addcol=: (<@[ ,.~ ])each
has=: +./@:rxE
NB. Get stats
Stats=:/:~@:(~.@:tractcol ,. tractcol <"0@(#,mean)/. tonum@:getcol&('num_fibers-num_fibers';'FA-mean';'FA-stDev')) each csvs
GroupedStats=./:~ ({."1 </. ]) ; (35&}. each REMOTE_PATHS) addcol Stats
Header=: ;: 'Bundle numCases numTractsAvg meanFaAvg stDevFAAvg InputCsv'
Result=. Header , }. ; a:&,each  (#~ 'cb|uf|af|slf|ilf|ioff'&has@:,@:(":@>)@>) GroupedStats
'wmql_comparison.csv' writecsv~ Result
Result

┌─────────────┬────────┬────────────┬─────────┬──────────┬───────────────────────────────────────────────┐
│Bundle       │numCases│numTractsAvg│meanFaAvg│stDevFAAvg│InputCsv                                       │
├─────────────┼────────┼────────────┼─────────┼──────────┼───────────────────────────────────────────────┤
│af.left      │10      │390.8       │695.219  │167.569   │original-matlabukf/tractmeasures.csv           │
├─────────────┼────────┼────────────┼─────────┼──────────┼───────────────────────────────────────────────┤
│af.left      │10      │472.9       │686.361  │167.255   │original-matlabukf-newquery/tractmeasures.csv  │
├─────────────┼────────┼────────────┼─────────┼──────────┼───────────────────────────────────────────────┤
│af.left      │10      │463.7       │649.323  │174.765   │original-cppukf/tractmeasures.csv              │
├─────────────┼────────┼────────────┼─────────┼──────────┼───────────────────────────────────────────────┤
│af.left      │10      │568.8       │